In [ ]:
from pyspark import SparkContext
from datetime import datetime

In [ ]:
sc = SparkContext(appName="Assignment_conditions")

In [ ]:
threshold=40

In [ ]:
data = sc.textFile("testing.csv")
data = data.map(lambda line: line.split(",")).filter(lambda v: v[0]!="START")
condition_map = data.map(lambda line: (int(line[4]),line[5])).distinct()#reduceByKey(lambda val1, val2: val1)
condition_map.take(5)

In [ ]:
format_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
condition_map.saveAsTextFile("{0}/{1}".format("condition_map", format_time))


In [ ]:
patient_rdd2 = data.map(lambda line: (line[2],line[4])).groupByKey().map(lambda tp: [int(j) for j in set(tp[1])])#.map(lambda k: (k[0],[int[j] for j in k[1]]))
n_baskets = patient_rdd2.count()
patient_rdd2.take(50)

In [ ]:

frequent_singles = patient_rdd2.flatMap(lambda l: [(i,1) for i in l]).reduceByKey(lambda v1,v2:v1+v2).sortBy(lambda a:a[1],False).filter(lambda k: k[1]>threshold)
frequent_singles.take(10)

In [ ]:
freq_single_item_set = set([k[0] for k in frequent_singles.collect()])
print(freq_single_item_set)

In [ ]:
def pair_forming(list):
    pairs=[]
    list.sort()
    for i in range(0,len(list)):
        if list[i] in freq_single_item_set:
            for j in range(i+1,len(list)):
                if  list[i]!=list[j] and list[j] in freq_single_item_set:
                    pairs.append((list[i],list[j]))
    return pairs

In [ ]:
def triple_forming(list):
    triples=[]
    pairs = pair_forming(list)
    for i,j in pairs:
        for k in list:
            if (i,j) in frequent_pairs and k in freq_single_item_set and k!=i and k!=j:#or (j,i) in frequent_pairs#prob unecessary
                  triples.append(tuple(sorted([i,j,k])))
    return triples

In [ ]:
###COUNTING PAIRS
pair_count = patient_rdd2.flatMap(pair_forming).map(lambda p:(p,1)).reduceByKey(lambda v1,v2: v1+v2).filter(lambda tp:tp[1]>=threshold)
pair_count.sortBy(lambda a: a[1],False).take(10)

In [ ]:
###FREQUENT PAIR SET
frequent_pairs = set(pair_count.map(lambda tp:tp[0]).collect())
print(frequent_pairs)

In [ ]:
###COUNTING TRIPLES
triple_count = patient_rdd2.flatMap(triple_forming).map(lambda p:(p,1)).reduceByKey(lambda v1,v2: v1+v2).filter(lambda tp:tp[1]>=threshold)
triple_count.sortBy(lambda a: a[1], False).take(10)

In [ ]:
###FREQUENT TRIPLE SET
frequent_triples = set(triple_count.map(lambda tp:tp[0]).collect())
print(frequent_triples)

In [ ]:
min_std_lift = 0.2
n_baskets

In [ ]:
def confidence_1to1(item1, item2):
    pass
